In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os

In [187]:
QQQ = pd.read_csv("/Users/vladimirbarshchuk/Downloads/QQQ.csv", parse_dates=True, index_col=0)
QQQ["Close"].head()

Date
1999-03-10    51.0625
1999-03-11    51.3125
1999-03-12    50.0625
1999-03-15    51.5000
1999-03-16    51.9375
Name: Close, dtype: float64

In [188]:
QQQ['QQQ']=(QQQ['Close'].pct_change())
QQQ.drop(columns = ["Open", "High","Low","Close", "Adj Close", "Volume"], inplace=True)

In [190]:
# Get the current working directory. I need the current working directory so that I am able to upload the Fama French dataset.
cwd = os.getcwd()
cwd

'/Users/vladimirbarshchuk/Downloads'

In [61]:
#Now in order to use the Fama-French Model dataset, I would have to change the directory to the one where my dataset is saved (Downloads folder).
os.chdir('/Users/vladimirbarshchuk/Downloads')
os.getcwd()

In [50]:
#QQQ data is given in daily returns, I would have to convert this to monthly using the model which was developed in 
#edhec course earlier...

In [191]:
QQQ_m = QQQ.resample('M').apply(erk.compound).to_period("M")
QQQ_m.head()

,QQQ
Date,
1999-03,0.027540
1999-04,0.023824
1999-05,-0.031414
1999-06,0.107508
1999-07,-0.017896


In [192]:
fff = pd.read_csv("/Users/vladimirbarshchuk/Downloads/F-F_Research_Data_Factors_m.csv")
fff.columns

Index(['Unnamed: 0', 'Mkt-RF', 'SMB', 'HML', 'RF'], dtype='object')

In [62]:
#We would have to rename column Unnamed to Date and set it as the index...

In [193]:
fff.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
fff.set_index("Date", inplace =True)

In [194]:
# Now let's run the regression, I am going to be using the OLS function from statsmodel package.
#I take QQQ from 1999-03 to 2022-01 and FamaFrench for the same period. 
#Next I regress QQQ_excess returns on the Mkt_excess returns to check the relationship.
#I have to reshape Y so OLS function runs. 

QQQ_excess= QQQ_m["1999-03":"2022-01"]-fff.loc["199903":"202201", ["RF"]].values
mkt_excess = fff.loc["199903":"202201", ["Mkt-RF"]]
exp_var = mkt_excess.copy()
exp_var["Constant"] = 1
y=QQQ_excess.values.reshape(-1,1)
x=exp_var

In [195]:
lm = sm.OLS(y,x).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     70.92
Date:                Tue, 08 Nov 2022   Prob (F-statistic):           2.14e-15
Time:                        18:03:15   Log-Likelihood:                 121.17
No. Observations:                 275   AIC:                            -238.3
Df Residuals:                     273   BIC:                            -231.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0176      0.002      8.421      0.000       0.014       0.022
Constant      -0.1350      0.010    -14.187      0.000      -0.154      -0.116
==============================================================================
Omnibus:                       35.890   Durbin-Watson:                   0.138
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.222
Skew:                          -1.014   Prob(JB):                     5.57e-11
Kurtosis:                       3.088   Cond. No.                         4.59
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [196]:
# Now I follow the same procedure with the HML aka Value and SMB aka Size.
exp_var["Value"]=fff.loc["199903":"202201", ["HML"]]
exp_var["Size"] = fff.loc["199903":"202201", ["SMB"]]

In [197]:
lm = sm.OLS(y,x).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     29.02
Date:                Tue, 08 Nov 2022   Prob (F-statistic):           2.67e-16
Time:                        18:03:53   Log-Likelihood:                 127.72
No. Observations:                 275   AIC:                            -247.4
Df Residuals:                     271   BIC:                            -233.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0174      0.002      8.194      0.000       0.013       0.022
Constant      -0.1337      0.009    -14.303      0.000      -0.152      -0.115
Value         -0.0098      0.003     -3.605      0.000      -0.015      -0.004
Size          -0.0008      0.003     -0.263      0.793      -0.007       0.005
==============================================================================
Omnibus:                       31.974   Durbin-Watson:                   0.121
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.795
Skew:                          -0.937   Prob(JB):                     1.38e-09
Kurtosis:                       2.784   Cond. No.                         4.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""